In [4]:
%load_ext autoreload
%autoreload 2

In [6]:
import gpxpy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
from gps_analysis import geodesy, files, splits, garmin

In [50]:
api = garmin.login("imogen@gtsoftware.co.uk")

Input your Garmin password: ········


In [53]:
activities = garmin.get_activities(0, 10, activityType='rowing')

In [54]:
activity_data, errors = garmin.load_activities(activities.activityId)

  0%|          | 0/10 [00:00<?, ?it/s]

In [55]:
garmin.activity_data_to_excel(activities, activity_data)

In [ ]:
locations = {
    'start': [51.461398172524824, -0.9455592979289794],
    '500m': [51.46460471321316, -0.9404994529495201],
    '1km': [51.467811036115336, -0.9354388968526993],
    '1.5km': [51.47101714117572, -0.930377629505215], 
    'finish': [51.47420259934778, -0.9253479107461384],
}
locations = pd.DataFrame.from_dict({
    k: {
        'latitude': v[0], 
        'longitude': v[1],
        'bearing': geodesy.estimate_bearing(
            positions, geodesy.LatLon(*v), 0.05)
    } 
    for k, v in locations.items()
},
    orient='index')
locations.index.name='location'
locations.bearing = geodesy.bearing(
    locations.loc['start'],
    locations.loc['finish']
)

In [ ]:
locations.to_csv("data/cav_locations.tsv", sep='\t')

In [ ]:
actid = 8026393665
actid = 2366290503

positions = garmin.load_activity(actid, api=api)

In [ ]:
garmin.login('imogen@gtsoftware.co.uk')
actid = 2366290503
data = garmin.load_fit_activity(actid)

In [ ]:
data.time.diff(-1)

In [ ]:
best_splits = splits.find_all_best_times(data)

In [ ]:
best_splits.loc["2km"]

In [ ]:
def interval_average_value(locations, label, start_dist, dist):
    interval = [start_dist, start_dist + dist]
    i0, i1 = locations.distance.searchsorted(interval)
    val = locations.loc[i0:i1, label]
    times = locations.timeElapsed.dt.total_seconds().copy()
    times.loc[[i0, i1 + 1]] = np.interp(interval, locations.distance, times)
    weights = -times.loc[i0:i1+1].diff(-1)[:-1]
    return np.average(val, weights=weights)

interval_average_value(data, 'cadence', 12.041, 2)

In [ ]:
interval_average_value(data, 'cadence', 3.4, 2)

In [ ]:
data.distance.diff(-1)

In [ ]:
plt.figure()
plt.plot(data.distance, data.cadence)

In [ ]:
distance = 2
dist_splits = best_splits.loc['2km']

plt.figure()
plt.plot(data.distance, 500/data.speed)
plt.plot(data.distance, data.heart_rate)
plt.plot(data.distance, data.cadence)

for d, s in dist_splits.split.iteritems():
    ts = s.total_seconds()
    plt.plot([d, d+distance], [ts, ts])
    
plt.ylim(0, 250)

In [ ]:
activities = garmin.get_activities(activityType='rowing', api=api)
activity_data, errors = garmin.load_activities(activities.activityId, api=api)
garmin.activity_data_to_excel(activities, activity_data, api=api)

In [ ]:
activities = garmin.get_activities(activityType='rowing', api=api)
activity_data, errors = garmin.load_activities(activities.activityId, api=api)